In [32]:
# import pandas_ta as ta
import pandas as pd
import pandas_ta as ta
import numpy as np
from datetime import datetime, timedelta
import plotly.graph_objects as go
import warnings
# from binance.client import Client   # Load historical price data from binance exchange 
# client = Client()
warnings.filterwarnings("ignore", category=FutureWarning)
import asyncio
import nest_asyncio
nest_asyncio.apply()
import asyncio
from binance import AsyncClient
import time
import myUtils
from botConfig import *
# from myUtils import *
pd.set_option('display.max_rows', 250)

In [78]:
import importlib
# If you want to update the module:
importlib.reload(myUtils)
from myUtils import *

In [10]:
cb = ccxtBinance()

cb.binanceActivate(mode)

starting = time.time()
print(cb.getqty('USDT'))

Authentication Successful.
10000.0
0.19493865966796875


In [21]:
coins = pd.read_csv('spot_coins_list.csv').loc[:,'0'].tolist()
coins

['BTCUSDT',
 'ETHUSDT',
 'BNBUSDT',
 'BCCUSDT',
 'NEOUSDT',
 'LTCUSDT',
 'QTUMUSDT',
 'ADAUSDT',
 'XRPUSDT',
 'EOSUSDT',
 'TUSDUSDT',
 'IOTAUSDT',
 'XLMUSDT',
 'ONTUSDT',
 'TRXUSDT',
 'ETCUSDT',
 'ICXUSDT',
 'VENUSDT',
 'NULSUSDT',
 'VETUSDT',
 'PAXUSDT',
 'BCHABCUSDT',
 'BCHSVUSDT',
 'USDCUSDT',
 'LINKUSDT',
 'WAVESUSDT',
 'BTTUSDT',
 'USDSUSDT',
 'ONGUSDT',
 'HOTUSDT',
 'ZILUSDT',
 'ZRXUSDT',
 'FETUSDT',
 'BATUSDT',
 'XMRUSDT',
 'ZECUSDT',
 'IOSTUSDT',
 'CELRUSDT',
 'DASHUSDT',
 'NANOUSDT',
 'OMGUSDT',
 'THETAUSDT',
 'ENJUSDT',
 'MITHUSDT',
 'MATICUSDT',
 'ATOMUSDT',
 'TFUELUSDT',
 'ONEUSDT',
 'FTMUSDT',
 'ALGOUSDT',
 'USDSBUSDT',
 'GTOUSDT',
 'ERDUSDT',
 'DOGEUSDT',
 'DUSKUSDT',
 'ANKRUSDT',
 'WINUSDT',
 'COSUSDT',
 'NPXSUSDT',
 'COCOSUSDT',
 'MTLUSDT',
 'TOMOUSDT',
 'PERLUSDT',
 'DENTUSDT',
 'MFTUSDT',
 'KEYUSDT',
 'STORMUSDT',
 'DOCKUSDT',
 'WANUSDT',
 'FUNUSDT',
 'CVCUSDT',
 'CHZUSDT',
 'BANDUSDT',
 'BUSDUSDT',
 'BEAMUSDT',
 'XTZUSDT',
 'RENUSDT',
 'RVNUSDT',
 'HCUSDT',
 'HBARUSD

In [61]:
timeframe = "1m"
start = "2023-12-01"
# start_date = datetime.strptime(start, "%Y-%m-%d")
end = str(datetime.now())
# Calculate the difference in days
# day = (datetime.now() - start_date).days

# OTHER PARAMETERS
fast_ema_period=9
slow_ema_period=18

usdt_amount = 10 # usdt amount allocated for each bot

tp_perc= 10

In [53]:
starting = time.time()

signals_dict = {'coins' : [], 'buy_signals' : [], 'sell_signals' : []}
async def fetch_ohlcv(coin):
    try:
        data = await getdata(coin, timeframe)
        print(coin, data)
        signals_dict['coins'].append(coin)
        signals_dict['buy_signals'].append(data.buy_signal.iloc[-2])
        signals_dict['sell_signals'].append(data.sell_signal.iloc[-2])
    except Exception as e:
        print(coin, e)
        return None
    return data

async def fetch_all_ohlcv(coins):

    tasks = [fetch_ohlcv(coin) for coin in coins]
    results = await asyncio.gather(*tasks)

    return results

async def main():
    ohlcv_data = []
    client = await AsyncClient.create()
    ohlcv_data = await fetch_all_ohlcv(coins)
    await client.close_connection()
    return ohlcv_data


data = asyncio.run(main())
    
ending = time.time()

total_time = ending - starting
print("It took {} seconds to make {} API calls".format(total_time, len(coins)))

XRPUSDT                        Open    High     Low   Close    Volume  fast_ema  \
timestamp                                                                 
2023-12-31 19:12:00  0.6214  0.6214  0.6212  0.6213   69829.0       NaN   
2023-12-31 19:13:00  0.6212  0.6213  0.6211  0.6211   13425.0       NaN   
2023-12-31 19:14:00  0.6211  0.6212  0.6210  0.6211   25646.0       NaN   
2023-12-31 19:15:00  0.6211  0.6212  0.6210  0.6211   31266.0       NaN   
2023-12-31 19:16:00  0.6212  0.6213  0.6211  0.6212   54513.0       NaN   
2023-12-31 19:17:00  0.6212  0.6213  0.6211  0.6211  265231.0       NaN   
2023-12-31 19:18:00  0.6211  0.6211  0.6210  0.6210   19304.0       NaN   
2023-12-31 19:19:00  0.6210  0.6212  0.6210  0.6212   51446.0       NaN   
2023-12-31 19:20:00  0.6212  0.6212  0.6211  0.6211   13169.0  0.621133   
2023-12-31 19:21:00  0.6212  0.6212  0.6205  0.6205  102855.0  0.621007   
2023-12-31 19:22:00  0.6206  0.6206  0.6202  0.6205   67039.0  0.620905   
2023-12-31 19:23:

WAVESUSDT                       Open   High    Low  Close    Volume  fast_ema  slow_ema  \
timestamp                                                                       
2023-12-31 19:12:00  2.728  2.729  2.728  2.729    842.74       NaN       NaN   
2023-12-31 19:13:00  2.730  2.730  2.728  2.729   3261.69       NaN       NaN   
2023-12-31 19:14:00  2.728  2.728  2.728  2.728    105.49       NaN       NaN   
2023-12-31 19:15:00  2.728  2.730  2.728  2.729   2167.40       NaN       NaN   
2023-12-31 19:16:00  2.730  2.730  2.729  2.729    633.51       NaN       NaN   
2023-12-31 19:17:00  2.728  2.728  2.727  2.728     54.63       NaN       NaN   
2023-12-31 19:18:00  2.728  2.728  2.720  2.720  11290.38       NaN       NaN   
2023-12-31 19:19:00  2.721  2.724  2.721  2.723   1027.82       NaN       NaN   
2023-12-31 19:20:00  2.722  2.723  2.722  2.723     85.43  2.726444       NaN   
2023-12-31 19:21:00  2.723  2.723  2.720  2.720   1321.26  2.725156       NaN   
2023-12-31 19:22:0

IOSTUSDT                         Open     High      Low    Close     Volume  fast_ema  \
timestamp                                                                      
2023-12-31 19:12:00  0.01011  0.01011  0.01010  0.01010    94373.0       NaN   
2023-12-31 19:13:00  0.01011  0.01012  0.01011  0.01012  1456927.0       NaN   
2023-12-31 19:14:00  0.01012  0.01012  0.01012  0.01012    85625.0       NaN   
2023-12-31 19:15:00  0.01012  0.01012  0.01012  0.01012     2974.0       NaN   
2023-12-31 19:16:00  0.01012  0.01012  0.01012  0.01012    12547.0       NaN   
2023-12-31 19:17:00  0.01012  0.01012  0.01012  0.01012     2949.0       NaN   
2023-12-31 19:18:00  0.01011  0.01012  0.01010  0.01011   101321.0       NaN   
2023-12-31 19:19:00  0.01011  0.01011  0.01011  0.01011        0.0       NaN   
2023-12-31 19:20:00  0.01012  0.01012  0.01012  0.01012    33103.0  0.010116   
2023-12-31 19:21:00  0.01011  0.01011  0.01011  0.01011      593.0  0.010114   
2023-12-31 19:22:00  0.01011  0

HCUSDT                        Open    High     Low   Close  Volume  fast_ema  \
timestamp                                                               
2020-11-10 11:40:00  0.8470  0.8470  0.8470  0.8470    0.00       NaN   
2020-11-10 11:41:00  0.8476  0.8476  0.8476  0.8476  285.12       NaN   
2020-11-10 11:42:00  0.8476  0.8480  0.8470  0.8480  781.42       NaN   
2020-11-10 11:43:00  0.8480  0.8480  0.8474  0.8474  866.72       NaN   
2020-11-10 11:44:00  0.8496  0.8496  0.8474  0.8474   27.82       NaN   
2020-11-10 11:45:00  0.8496  0.8496  0.8496  0.8496   12.00       NaN   
2020-11-10 11:46:00  0.8496  0.8496  0.8496  0.8496    0.00       NaN   
2020-11-10 11:47:00  0.8496  0.8496  0.8496  0.8496    0.00       NaN   
2020-11-10 11:48:00  0.8496  0.8496  0.8496  0.8496    0.00  0.848422   
2020-11-10 11:49:00  0.8496  0.8496  0.8496  0.8496    0.00  0.848658   
2020-11-10 11:50:00  0.8476  0.8476  0.8474  0.8474  647.08  0.848406   
2020-11-10 11:51:00  0.8474  0.8474  0.8474 

DREPUSDT                        Open    High     Low   Close   Volume  fast_ema  \
timestamp                                                                
2023-12-31 19:12:00  0.2859  0.2860  0.2856  0.2856    293.0       NaN   
2023-12-31 19:13:00  0.2858  0.2862  0.2858  0.2859   1394.0       NaN   
2023-12-31 19:14:00  0.2860  0.2866  0.2860  0.2866   6095.0       NaN   
2023-12-31 19:15:00  0.2863  0.2869  0.2857  0.2861  16967.0       NaN   
2023-12-31 19:16:00  0.2860  0.2860  0.2860  0.2860    109.0       NaN   
2023-12-31 19:17:00  0.2860  0.2860  0.2852  0.2859  11848.0       NaN   
2023-12-31 19:18:00  0.2856  0.2856  0.2855  0.2855    485.0       NaN   
2023-12-31 19:19:00  0.2859  0.2882  0.2859  0.2881  49801.0       NaN   
2023-12-31 19:20:00  0.2880  0.2890  0.2872  0.2876  38093.0  0.286367   
2023-12-31 19:21:00  0.2876  0.2876  0.2873  0.2873   1088.0  0.286553   
2023-12-31 19:22:00  0.2873  0.2873  0.2873  0.2873      0.0  0.286703   
2023-12-31 19:23:00  0.2877  

...

WRXUSDT                        Open    High     Low   Close   Volume  fast_ema  \
timestamp                                                                
2023-12-31 19:12:00  0.2298  0.2300  0.2298  0.2300    395.2       NaN   
2023-12-31 19:13:00  0.2297  0.2297  0.2297  0.2297    320.9       NaN   
2023-12-31 19:14:00  0.2297  0.2298  0.2296  0.2298    581.4       NaN   
2023-12-31 19:15:00  0.2296  0.2300  0.2296  0.2298   6450.3       NaN   
2023-12-31 19:16:00  0.2297  0.2298  0.2292  0.2292   9539.7       NaN   
2023-12-31 19:17:00  0.2291  0.2292  0.2287  0.2291   7320.5       NaN   
2023-12-31 19:18:00  0.2288  0.2293  0.2285  0.2285  11594.3       NaN   
2023-12-31 19:19:00  0.2287  0.2293  0.2286  0.2293   9108.9       NaN   
2023-12-31 19:20:00  0.2293  0.2293  0.2290  0.2290   1637.4  0.229378   
2023-12-31 19:21:00  0.2290  0.2293  0.2290  0.2290   5344.6  0.229302   
2023-12-31 19:22:00  0.2291  0.2292  0.2288  0.2288  13086.2  0.229202   
2023-12-31 19:23:00  0.2288  0

VGXUSDT                        Open    High     Low   Close  Volume  fast_ema  \
timestamp                                                               
2023-12-31 19:12:00  0.1386  0.1386  0.1386  0.1386  2131.6       NaN   
2023-12-31 19:13:00  0.1386  0.1386  0.1386  0.1386     0.0       NaN   
2023-12-31 19:14:00  0.1386  0.1386  0.1384  0.1384   507.2       NaN   
2023-12-31 19:15:00  0.1384  0.1384  0.1384  0.1384     0.0       NaN   
2023-12-31 19:16:00  0.1384  0.1384  0.1384  0.1384  7368.1       NaN   
2023-12-31 19:17:00  0.1384  0.1384  0.1384  0.1384     0.0       NaN   
2023-12-31 19:18:00  0.1386  0.1386  0.1386  0.1386   432.9       NaN   
2023-12-31 19:19:00  0.1386  0.1386  0.1385  0.1385  5272.7       NaN   
2023-12-31 19:20:00  0.1385  0.1385  0.1385  0.1385     0.0  0.138489   
2023-12-31 19:21:00  0.1386  0.1386  0.1386  0.1386  2254.3  0.138511   
2023-12-31 19:22:00  0.1385  0.1388  0.1385  0.1388  9897.0  0.138569   
2023-12-31 19:23:00  0.1388  0.1388  0.1388

SHIBUSDT                          Open      High       Low     Close        Volume  \
timestamp                                                                   
2023-12-31 19:12:00  0.000011  0.000011  0.000011  0.000011  1.351024e+08   
2023-12-31 19:13:00  0.000011  0.000011  0.000011  0.000011  4.910621e+08   
2023-12-31 19:14:00  0.000011  0.000011  0.000011  0.000011  1.088425e+08   
2023-12-31 19:15:00  0.000011  0.000011  0.000011  0.000011  1.109092e+09   
2023-12-31 19:16:00  0.000011  0.000011  0.000011  0.000011  1.195597e+08   
2023-12-31 19:17:00  0.000011  0.000011  0.000011  0.000011  5.616535e+06   
2023-12-31 19:18:00  0.000011  0.000011  0.000011  0.000011  1.333434e+08   
2023-12-31 19:19:00  0.000011  0.000011  0.000011  0.000011  3.190309e+08   
2023-12-31 19:20:00  0.000011  0.000011  0.000011  0.000011  6.886639e+08   
2023-12-31 19:21:00  0.000011  0.000011  0.000010  0.000011  6.929568e+09   
2023-12-31 19:22:00  0.000011  0.000011  0.000010  0.000010  1.3713

LINAUSDT                          Open      High       Low     Close      Volume  \
timestamp                                                                 
2023-12-31 19:12:00  0.011091  0.011092  0.011082  0.011082   633431.01   
2023-12-31 19:13:00  0.011088  0.011089  0.011084  0.011085   216950.67   
2023-12-31 19:14:00  0.011083  0.011089  0.011076  0.011076  1029173.56   
2023-12-31 19:15:00  0.011082  0.011083  0.011068  0.011070   733213.52   
2023-12-31 19:16:00  0.011068  0.011072  0.011067  0.011067    37269.40   
2023-12-31 19:17:00  0.011065  0.011066  0.011060  0.011065   112319.38   
2023-12-31 19:18:00  0.011068  0.011073  0.011065  0.011065    63139.15   
2023-12-31 19:19:00  0.011061  0.011079  0.011061  0.011079   104130.58   
2023-12-31 19:20:00  0.011077  0.011080  0.011069  0.011080    77105.84   
2023-12-31 19:21:00  0.011082  0.011088  0.011072  0.011072   279483.46   
2023-12-31 19:22:00  0.011072  0.011091  0.011072  0.011091   219938.60   
2023-12-31 19:23

AERGOUSDT                        Open    High     Low   Close  Volume  fast_ema  \
timestamp                                                               
2023-12-31 19:12:00  0.1489  0.1489  0.1489  0.1489     0.0       NaN   
2023-12-31 19:13:00  0.1489  0.1489  0.1489  0.1489     0.0       NaN   
2023-12-31 19:14:00  0.1489  0.1489  0.1489  0.1489     0.0       NaN   
2023-12-31 19:15:00  0.1492  0.1492  0.1492  0.1492    72.0       NaN   
2023-12-31 19:16:00  0.1492  0.1492  0.1492  0.1492     0.0       NaN   
2023-12-31 19:17:00  0.1492  0.1493  0.1492  0.1492   760.0       NaN   
2023-12-31 19:18:00  0.1492  0.1492  0.1492  0.1492     0.0       NaN   
2023-12-31 19:19:00  0.1492  0.1492  0.1492  0.1492     0.0       NaN   
2023-12-31 19:20:00  0.1492  0.1492  0.1492  0.1492     0.0  0.149100   
2023-12-31 19:21:00  0.1492  0.1492  0.1492  0.1492     0.0  0.149120   
2023-12-31 19:22:00  0.1491  0.1491  0.1491  0.1491    35.0  0.149116   
2023-12-31 19:23:00  0.1491  0.1491  0.14

In [55]:
signals_df = pd.DataFrame(signals_dict).set_index('coins')
signals_df

,buy_signals,sell_signals
coins,,
XRPUSDT,False,False
STORMUSDT,False,False
CVCUSDT,False,False
KEYUSDT,False,False
CHZUSDT,False,False
...,...,...
USTCUSDT,False,False
OAXUSDT,False,False
ENSUSDT,True,False


In [58]:
buysignals = signals_df[signals_df['buy_signals'] == True].index.to_list()
sellsignals = signals_df[signals_df['sell_signals'] == True].index.to_list()
buysignals

['CAKEUSDT',
 'FIROUSDT',
 'CTXCUSDT',
 'PAXUSDT',
 'LITUSDT',
 'DCRUSDT',
 'CVXUSDT',
 'ACHUSDT',
 'AVAUSDT',
 'REIUSDT',
 'BURGERUSDT',
 'ENSUSDT']

In [59]:
sellsignals

['UFTUSDT', 'AMBUSDT', 'FXSUSDT', 'ZENUSDT', 'SNXUSDT', 'ARKUSDT']

## BOT

In [72]:
cb.place_sell_order(coins[0], 0.0002)

{'info': {'symbol': 'BTCUSDT',
  'orderId': '14810675',
  'orderListId': '-1',
  'clientOrderId': 'x-R4BD3S8290abb66a1ebae2979f4e39',
  'transactTime': '1704052457565',
  'price': '0.00000000',
  'origQty': '0.00020000',
  'executedQty': '0.00020000',
  'cummulativeQuoteQty': '8.53255000',
  'status': 'FILLED',
  'timeInForce': 'GTC',
  'type': 'MARKET',
  'side': 'SELL',
  'workingTime': '1704052457565',
  'fills': [{'price': '42662.75000000',
    'qty': '0.00020000',
    'commission': '0.00000000',
    'commissionAsset': 'USDT',
    'tradeId': '2448601'}],
  'selfTradePreventionMode': 'EXPIRE_MAKER'},
 'id': '14810675',
 'clientOrderId': 'x-R4BD3S8290abb66a1ebae2979f4e39',
 'timestamp': 1704052457565,
 'datetime': '2023-12-31T19:54:17.565Z',
 'lastTradeTimestamp': 1704052457565,
 'lastUpdateTimestamp': 1704052457565,
 'symbol': 'BTC/USDT',
 'type': 'market',
 'timeInForce': 'GTC',
 'postOnly': False,
 'reduceOnly': None,
 'side': 'sell',
 'price': 42662.75,
 'triggerPrice': None,
 'a

In [73]:
cb.calculate_order_size(coins[0], usdt_amount)

0.0002344403136061187

In [80]:
starting = time.time()

trades_dict = {'coins' : [], 'orderData' : []}

async def buyCoin(coin):
    try:
        orderSize = cb.calculate_order_size(coin, usdt_amount)
        print(coin, orderSize)
        order = cb.place_buy_order(coin, orderSize)
        trades_dict['coins'].append(coin)
        trades_dict['orderData'].append(order)
    except Exception as e:
        print(coin, e)
        return None
    return data

async def buy_all_coins(buysignals):

    tasks = [buyCoin(coin) for coin in buysignals]
    results = await asyncio.gather(*tasks)

    return results

async def main():
    ohlcv_data = []
    client = await AsyncClient.create()
    ohlcv_data = await buy_all_coins(buysignals)
    await client.close_connection()
    return ohlcv_data

data = asyncio.run(main())

data = asyncio.run(main())
    
ending = time.time()

total_time = ending - starting
print("It took {} seconds to make {} API calls".format(total_time, len(buysignals)))

CAKEUSDT 2.8129395218002813
FIROUSDT 4.389815627743634
CTXCUSDT 22.967386311437757
PAXUSDT binance does not have market symbol PAXUSDT
LITUSDT 9.569377990430622
DCRUSDT 0.5461496450027308
CVXUSDT 2.960331557134399
ACHUSDT 457.2473708276177
AVAUSDT 15.408320493066256
REIUSDT 302.8467595396729
BURGERUSDT 16.866250632484398
ENSUSDT 1.002004008016032
CAKEUSDT 2.815315315315315
FIROUSDT 4.3878894251864855
CTXCUSDT 22.93577981651376
PAXUSDT binance does not have market symbol PAXUSDT
LITUSDT 9.569377990430622
DCRUSDT 0.5476451259583789
CVXUSDT 2.962962962962963
ACHUSDT 456.62100456621005
AVAUSDT 15.384615384615383
REIUSDT 302.8467595396729
BURGERUSDT 16.866250632484398
ENSUSDT 1.0050251256281408
It took 10.677606344223022 seconds to make 12 API calls


In [81]:
trades_dict

{'coins': ['CAKEUSDT',
  'FIROUSDT',
  'CTXCUSDT',
  'LITUSDT',
  'DCRUSDT',
  'CVXUSDT',
  'ACHUSDT',
  'AVAUSDT',
  'REIUSDT',
  'BURGERUSDT',
  'ENSUSDT',
  'CAKEUSDT',
  'FIROUSDT',
  'CTXCUSDT',
  'LITUSDT',
  'DCRUSDT',
  'CVXUSDT',
  'ACHUSDT',
  'AVAUSDT',
  'REIUSDT',
  'BURGERUSDT',
  'ENSUSDT'],
 'orderData': [{'info': {'symbol': 'CAKEUSDT',
    'orderId': '2017453',
    'orderListId': '-1',
    'clientOrderId': 'x-R4BD3S8240527034369c2eb7ddf02',
    'transactTime': '1704052894633',
    'price': '0.00000000',
    'origQty': '2.81000000',
    'executedQty': '2.81000000',
    'cummulativeQuoteQty': '9.98112000',
    'status': 'FILLED',
    'timeInForce': 'GTC',
    'type': 'MARKET',
    'side': 'BUY',
    'workingTime': '1704052894633',
    'fills': [{'price': '3.55200000',
      'qty': '2.81000000',
      'commission': '0.00000000',
      'commissionAsset': 'CAKE',
      'tradeId': '106815'}],
    'selfTradePreventionMode': 'EXPIRE_MAKER'},
   'id': '2017453',
   'clientOrder